In [1]:
from typing import List
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models

import weight_manager
# https://www.geeksforgeeks.org/ordereddict-in-python/

In [2]:
# simple model creation

model = tf.keras.Sequential()
model.add(layers.Dense(units=10, input_shape=(1,)))
model.add(layers.Dense(units=10))
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)


In [3]:
# automatic name creation
model.layers[0].name,model.layers[1].name

('dense', 'dense_1')

In [4]:
model.layers[0].get_weights() # gives (weights, bias)

[array([[ 0.14982563, -0.0463444 ,  0.39851207,  0.52118665,  0.00792533,
         -0.6418941 ,  0.4657076 , -0.69821626,  0.06772184, -0.12232101]],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]

In [5]:
f'{2+2}'

'4'

In [6]:
weight_manager.model_state_dict(model)

OrderedDict([('config',
              {'name': 'sequential',
               'layers': [{'class_name': 'InputLayer',
                 'config': {'batch_input_shape': (None, 1),
                  'dtype': 'float32',
                  'sparse': False,
                  'ragged': False,
                  'name': 'dense_input'}},
                {'class_name': 'Dense',
                 'config': {'name': 'dense',
                  'trainable': True,
                  'dtype': 'float32',
                  'batch_input_shape': (None, 1),
                  'units': 10,
                  'activation': 'linear',
                  'use_bias': True,
                  'kernel_initializer': {'class_name': 'GlorotUniform',
                   'config': {'seed': None}},
                  'bias_initializer': {'class_name': 'Zeros', 'config': {}},
                  'kernel_regularizer': None,
                  'bias_regularizer': None,
                  'activity_regularizer': None,
                  'ke

In [7]:
optimizer.get_config() # OK, et 'state ?'

{'name': 'Adam',
 'weight_decay': None,
 'clipnorm': None,
 'global_clipnorm': None,
 'clipvalue': None,
 'use_ema': False,
 'ema_momentum': 0.99,
 'ema_overwrite_frequency': None,
 'jit_compile': False,
 'is_legacy_optimizer': False,
 'learning_rate': 0.001,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

In [8]:
optimizer.get_weights()

AttributeError: 'Adam' object has no attribute 'get_weights'

In [ ]:
model.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_input'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 1),
    'units': 10,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_init

In [ ]:
model.get_weights() #list

[array([[ 0.52812535,  0.58084637,  0.4990613 ,  0.22483188,  0.01013732,
          0.5102213 ,  0.45367712, -0.41657427,  0.60514325,  0.3534482 ]],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([[-0.13399687,  0.22810459, -0.23489746, -0.20570233, -0.13280734,
         -0.24015099, -0.42723614, -0.38021898, -0.48436987,  0.02161139],
        [-0.17027634, -0.49828747,  0.510682  , -0.50270367, -0.33936083,
         -0.34350333,  0.06861341, -0.00315577,  0.18881047, -0.24222353],
        [ 0.3234129 ,  0.2960328 , -0.33112967, -0.05933845, -0.4949498 ,
          0.07043171, -0.165147  ,  0.23581094, -0.1767151 ,  0.05715686],
        [ 0.23700541,  0.01594025, -0.23426735, -0.05245233,  0.32681847,
          0.32907343, -0.36847264,  0.36963165, -0.5154205 ,  0.5077059 ],
        [-0.06984159, -0.38263458,  0.0040282 , -0.47909832, -0.00114214,
         -0.25650206,  0.3830313 , -0.37703043, -0.23080772,  0.14324886],
        [ 0.07780